In [ ]:
# Funktion berechnet gaußsche Fehlerfortpflanzung
#  funktion: Gesamtfunktion, aus der das gewünschte berechnet wird
#  variablen: Sympy Liste von variablennamen, nach denen die Funktion abgeleitet wird/ deren Fehler einfließen
#  variablen_werte: Liste gemessener (Mittel-)Werte aller Variablen, die auch in der variablen-Liste stehen
#  fehler_werte: Liste unkorrelierter statistischer oder systematischer Fehlerwerte der variablen-Liste

def gauss(funktion,variablen,variablen_werte,fehler_werte): 
    fehler = 0
    fehlersymbole=[]
    for var in variablen:                           # in dieser Schleife wird die Gaußsche Fehlerfortpflanzung mit Variablen erstellt
        d = sp.symbols('d' + var.name)              # Symbole fuer die Fehler werden generiert
        fehlersymbole.append(d)                     # Fehlersymbole werden in Liste eintragen
        partial = sp.diff(funktion, var) * d        # Partielle Differentation und mit mit Fehlersymbol 'd' multiplizieren
        fehler = fehler + partial**2                # Quadratsumme wird berechnet
    err1=sp.Subs(fehler,variablen,variablen_werte).doit()         # Variablenwerte (Zahlen) in FehlerFormel einsetzen 
    err2=sp.Subs(err1,fehlersymbole,fehler_werte).doit()          # Fehlerwerte (Zahlen) in FehlerFormel einsetzen und
                                                                  # Funktionswert berechnen
    return(sp.sqrt(err2))                           

# Funktion berechnet korrelierte statistische Unsicherheiten:
#  matrix_werte: numpy-Matrix aller Messwerte (keine Mittelwerte, siehe Beispiel) 
# benötigt keine Fehlerwerte!

def korreliert(funktion,variablen,variablen_werte,matrix_werte): #korrelierte statistische Unsicherheit
    fehler = 0
    V = np.cov(matrix_werte)          # Covarianzmatrix berechnen
    for var1 in variablen:                          
        for var2 in variablen:
            partial = sp.diff(funktion, var1)*sp.diff(funktion,var2) * V[variablen.index(var1)][variablen.index(var2)]   
            fehler = fehler + partial
    err1=sp.Subs(fehler,variablen,variablen_werte).doit()         #Variablenwerte (Zahlen) in FehlerFormel einsetzen 
    return(sp.sqrt(err1))

# Beispiel:
#import sympy as sp

#T0v,T1v,h0difv,h100difv, pav = sp.symbols('T0v T1v h0difv h100difv pav') 
#          neue Variablennamen im code nutzen, damit keine Werte eingesetzt werden
#          diese Form wird für die Ableitung nach bestimmten Zeichen verwendet

#variablen = [T0v,T1v,h0difv,h100difv,pav]  

#variablen_werte = [np.mean(T0),np.mean(T1),np.mean(h0dif),np.mean(h100dif),p_a]  
#          Mittelwerte (Reihenfolge wie Variablen)

#fehler_werte_sys = [0.1,0.1,0.001,0.001,50]                                      
#          systematische Fehlerwerte (Reihenfolge!)

#funktion = ((rho*h100difv-rho*h0difv)/((rho*h0difv+pav)*(T1v-T0v))+gamma*(rho*h100difv+pav)/(rho*h0difv+pav))
#          Gesamtfunktion für die berechnete Größe aufstellen
#          Variablennamen wie in der vorher erstellten variablenliste

#          Dies nur für korrelierte Fehlerrechung
#matrix_werte = np.array([T0,T1,h0dif,h100dif,np.ones(len(T0))*p_a])        
#          alle Messwerte, in gleichen Dimensionen (Anzahl an wiederholten Messungen) in die Matrix schreiben (auch unkorrelierte)